In [7]:
!pip install optuna
import optuna
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
df = pd.read_csv("/content/smart_grid_stability_augmented.csv")

In [4]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
y = np.where(y=="stable",1,-1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [9]:
def objective(trial, X=X_train, y=y_train, test_x = X_test, test_y = y_test):

    C = 10**trial.suggest_float("C", -4, 4)
    kernel = trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])
    degree = trial.suggest_int("degree", 2, 5)
    gamma = 10**trial.suggest_float("gamma", -4, 1)
    coef0 = trial.suggest_int("coef0", 1, 4)


    mod = SVC(C=C,
             kernel=kernel,
             degree=degree,
             gamma=gamma,
             coef0=coef0,
        )

    mod.fit(X,y)
    accuracy = accuracy_score(test_y,mod.predict(test_x))
    return accuracy


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2023-01-29 00:14:52,211] A new study created in memory with name: no-name-28c4e3d9-2550-46f4-ab5a-5f89d7b1fa28
[I 2023-01-29 00:18:05,591] Trial 0 finished with value: 0.9253333333333333 and parameters: {'C': -3.0929327693048387, 'kernel': 'sigmoid', 'degree': 4, 'gamma': -1.1271318914313566, 'coef0': 1}. Best is trial 0 with value: 0.9253333333333333.
[I 2023-01-29 00:18:35,619] Trial 1 finished with value: 0.9945833333333334 and parameters: {'C': 2.812625685806032, 'kernel': 'sigmoid', 'degree': 5, 'gamma': -2.8553370577754875, 'coef0': 3}. Best is trial 1 with value: 0.9945833333333334.
[I 2023-01-29 00:21:34,153] Trial 2 finished with value: 0.8953333333333333 and parameters: {'C': 0.9356122032145082, 'kernel': 'sigmoid', 'degree': 2, 'gamma': -3.591175006407742, 'coef0': 3}. Best is trial 1 with value: 0.9945833333333334.
[I 2023-01-29 00:22:51,996] Trial 3 finished with value: 0.9773333333333334 and parameters: {'C': 0.28262418459799044, 'kernel': 'sigmoid', 'degree': 5, 'gamm

In [10]:
trial = study.best_trial
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

Accuracy: 0.9995833333333334
Best hyperparameters: {'C': -0.01972321239857544, 'kernel': 'poly', 'degree': 2, 'gamma': 0.8732848623501065, 'coef0': 4}


In [11]:
optuna.visualization.plot_optimization_history(study)

In [12]:
optuna.visualization.plot_slice(study)